In [1]:
# prepare data
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os, json
import glob
import operator
from IPython.display import display, HTML

In [2]:
%load_ext autoreload
%autoreload 1

from shared_globals_functions import *
%aimport shared_globals_functions

In [3]:
t_name='t1'
merge_list=['t1.1', 't1.2', 't1.3', 't1.4']

cust_dir = 'test'

tdidf_max_df=1.0 # defaults to 1.0 (float), can be an int too
tdidf_min_df=4
use_tfidfvectorizer = True
use_stopwords = True
use_extended_stopwords = False

umap_metric='hellinger'
umap_ncomponents=2
umap_random_state=42
umap_min_dist=0 #0.1
umap_n_neighbors=30 #10, 30

min_cluster=10
max_cluster=30
clust_int=3

ss_min = 50
ss_max = 500
ss_int = 50
cs_min = 50
cs_max = 500
cs_int = 50

#tf-idf vectorizer
#max_df=0.3
#min_df=

#kmeans
#n_clusters
#n_init
#max_iter

In [4]:
# Parameters
cust_dir = "ar1/"
use_tfidfvectorizer = True
tdidf_max_df = 1.0
tdidf_min_df = 4
use_stopwords = True
use_extended_stopwords = False
umap_metric = "hellinger"
umap_ncomponents = 2
umap_random_state = 42
umap_min_dist = 0
umap_n_neighbors = 30
min_cluster = 10
max_cluster = 30
clust_int = 3
ss_min = 50
ss_max = 500
ss_int = 50
cs_min = 50
cs_max = 500
cs_int = 50
t_name = "t1"
merge_list = ["t1.1", "t1.2", "t1.3", "t1.4"]


In [5]:
testset = from_disk(t_name)

reading /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/t1.pkl


In [6]:
len(testset[testset['topics'].str.len() > 0])

69817

In [7]:
if len(merge_list)>0:
    t_name = f'{t_name}+{"+".join([re.sub(r"t", "", tid) for tid in merge_list])}'
    print(t_name)

t1+1.1+1.2+1.3+1.4


In [8]:
testset_merge = create_merge(testset, merge_list, merge_to_one=True)

reading /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/t1.1.pkl
reading /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/t1.2.pkl
reading /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/t1.3.pkl


reading /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/t1.4.pkl


In [9]:
len(testset_merge[testset_merge['topics'].str.len() > 0])

69817

In [10]:
df = testset_merge.merge(testset, on=['name'], how='left', indicator=True)

In [11]:
df[df['_merge']!='both']

,name,topics_x,topics_y,_merge


In [12]:
df[df['topics_x'].str.len() == 0]

,name,topics_x,topics_y,_merge


In [13]:
if len(merge_list)>0:
    testset = testset_merge

In [14]:
import pandas as pd
import umap
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [15]:
orig_testset = testset

In [16]:
my_stop_words=None
if use_stopwords:
    my_stop_words='english'
    if use_extended_stopwords:
        print("todo")

if use_tfidfvectorizer:
    vectorizer = TfidfVectorizer(min_df=tdidf_min_df, max_df=tdidf_max_df, stop_words=my_stop_words)
else:
    vectorizer = CountVectorizer(min_df=tdidf_min_df, max_df=tdidf_max_df, stop_words=my_stop_words)

print(f"total repos {len(testset[testset['topics'].str.len() > 0])}")
testset = testset[testset['topics'].str.len() > 0].reset_index()

if isinstance(testset['topics'][0], list):
    testset['topics'] = testset['topics'].apply(lambda x: ' '.join(x))
    print("topics came as list ... consider joining it before")

cr_start()
word_doc_matrix = vectorizer.fit_transform(testset['topics'])
runtime_vectorizer = cr_getrt()

total repos 69817
topics came as list ... consider joining it before


In [17]:
runtime_vectorizer

0.7587966918945312

In [18]:
word_doc_matrix

<69817x18666 sparse matrix of type '<class 'numpy.float64'>'
	with 690471 stored elements in Compressed Sparse Row format>

In [19]:
class ProgressWriter:
    def write(self, text):
        print("custom progress", n, total)

    def flush(self):
        pass

#tqdm_kwds = {"file": "progress_writer", "disable": False }
tqdm_kwds = {"disable": False }   


In [20]:
%%time
# try also with default parameters for n_neighbors and min_dist

cr_start()

embedding = umap.UMAP(
    n_neighbors=umap_n_neighbors,
    min_dist=umap_min_dist,
    n_components=umap_ncomponents, metric=umap_metric,
    tqdm_kwds=tqdm_kwds,
    random_state=umap_random_state,
).fit(word_doc_matrix)

runtime_umap = cr_getrt()

/home/manuel/d3tect-tools/venv/lib/python3.9/site-packages/umap/umap_.py:125: UserWarning: A few of your vertices were disconnected from the manifold.  This shouldn't cause problems.
Disconnection_distance = 1 has removed 34888 edges.
It has only fully disconnected 777 vertices.
Use umap.utils.disconnected_vertices() to identify them.
  warn(


Epochs completed:   0%|            0/200 [00:00]

CPU times: user 13min 8s, sys: 13.8 s, total: 13min 22s
Wall time: 3min 16s


In [21]:
print(runtime_umap)

196.31631231307983


In [22]:
%%time

cr_start()
clusterable_embedding = embedding.transform(word_doc_matrix)
runtime_umap_transform = cr_getrt()

/home/manuel/d3tect-tools/venv/lib/python3.9/site-packages/scipy/sparse/_index.py:146: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


Epochs completed:   0%|            0/30 [00:00]

CPU times: user 2min 37s, sys: 134 ms, total: 2min 37s
Wall time: 2min 17s


In [23]:
print(runtime_umap_transform)

137.97858548164368


In [24]:
umap_embedding = pd.DataFrame(embedding.embedding_, columns=['x', 'y']) # (umap_t)
umap_clusterable_embedding = pd.DataFrame(clusterable_embedding, columns=['x', 'y']) # (umap_t)

umap_embedding['name'] = testset['name']
umap_clusterable_embedding['name'] = testset['name']

to_disk_results(umap_embedding, f'umap_embedding_{t_name}', subdir=f"{cust_dir}")
to_disk_results(umap_clusterable_embedding, f'umap_clusterable_embedding_{t_name}', subdir=f"{cust_dir}")

writing /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/results/ar1//umap_embedding_t1+1.1+1.2+1.3+1.4.pkl
writing /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/results/ar1//umap_clusterable_embedding_t1+1.1+1.2+1.3+1.4.pkl


https://hdbscan.readthedocs.io/en/latest/parameter_selection.html

Since we have seen that min_samples clearly has a dramatic effect on clustering, the question becomes: how do we select this parameter? The simplest intuition for what min_samples does is provide a measure of how conservative you want you clustering to be. The larger the value of min_samples you provide, the more conservative the clustering – more points will be declared as noise, and clusters will be restricted to progressively more dense areas. We can see this in practice by leaving the min_cluster_size at 60, but reducing min_samples to 1.

In [25]:
import hdbscan

def clusterapply_hdbscan(hdbscan_min_samples, min_cluster_size, runner_name):
    cr_start()
    labels = hdbscan.HDBSCAN(
        min_samples=hdbscan_min_samples,
        min_cluster_size=min_cluster_size,
    ).fit_predict(clusterable_embedding)
    runtime_hdb = cr_getrt()
    
    stats = {'type': "hdb",
             'testset': t_name,
             'version': td_version,
            }
    
    detail_stats = {'hdb_minclust': min_cluster_size,
                    'hdb_minsamp': hdbscan_min_samples,
                    'runtime': runtime_hdb,
                    'tdf': f"rt_vect: {runtime_vectorizer}, min_df: {tdidf_min_df}, max_df: {tdidf_max_df}, stopwords: {use_stopwords}, extended stopwords: {use_extended_stopwords}",
                    'umap': f"{umap_metric}, rt_umap_tf: {runtime_umap_transform}, rt_umap: {runtime_umap}, ncomp: {umap_ncomponents}, rs: {umap_random_state}, min_dist: {umap_min_dist}, no_neigh: {umap_n_neighbors}",
                    #'gtr': grtr,
                    #'gtr_rel': grtr_rt
                   }
    prediction = stats
    prediction['name'] = runner_name
    #prediction.update(max_fscore(my_results[[tag, 'clabel']].apply(tuple, axis=1).to_dict()))
    #prediction.update({'tot_clust': total_clusters, 'tot_anom':no_anomalies, 'no_lab':missing_clabel, 'anom':identified_anom})
    prediction.update(detail_stats)
    res.append(prediction.copy())
    display(pd.DataFrame.from_dict(res))
    return labels

res = []
hdb_labels_results = pd.DataFrame()
hdb_labels_results['name'] = testset['name'].copy()


for cs in range(cs_min, cs_max, cs_int): #50, 500
    for ss in range(ss_min, ss_max, ss_int): #50 500
        runner_name = f'ss{ss}cs{cs}'
        hdb_labels_results[runner_name] = clusterapply_hdbscan(ss, cs, runner_name)

results = pd.DataFrame.from_dict(res)
res = []


,type,testset,version,name,hdb_minclust,hdb_minsamp,runtime,tdf,umap
0,hdb,t1+1.1+1.2+1.3+1.4,1,ss50cs50,50,50,1.688348,"rt_vect: 0.7587966918945312, min_df: 4, max_df...","hellinger, rt_umap_tf: 137.97858548164368, rt_..."


,type,testset,version,name,hdb_minclust,hdb_minsamp,runtime,tdf,umap
0,hdb,t1+1.1+1.2+1.3+1.4,1,ss50cs50,50,50,1.688348,"rt_vect: 0.7587966918945312, min_df: 4, max_df...","hellinger, rt_umap_tf: 137.97858548164368, rt_..."
1,hdb,t1+1.1+1.2+1.3+1.4,1,ss100cs50,50,100,1.372967,"rt_vect: 0.7587966918945312, min_df: 4, max_df...","hellinger, rt_umap_tf: 137.97858548164368, rt_..."


,type,testset,version,name,hdb_minclust,hdb_minsamp,runtime,tdf,umap
0,hdb,t1+1.1+1.2+1.3+1.4,1,ss50cs50,50,50,1.688348,"rt_vect: 0.7587966918945312, min_df: 4, max_df...","hellinger, rt_umap_tf: 137.97858548164368, rt_..."
1,hdb,t1+1.1+1.2+1.3+1.4,1,ss100cs50,50,100,1.372967,"rt_vect: 0.7587966918945312, min_df: 4, max_df...","hellinger, rt_umap_tf: 137.97858548164368, rt_..."
2,hdb,t1+1.1+1.2+1.3+1.4,1,ss150cs50,50,150,1.591077,"rt_vect: 0.7587966918945312, min_df: 4, max_df...","hellinger, rt_umap_tf: 137.97858548164368, rt_..."


,type,testset,version,name,hdb_minclust,hdb_minsamp,runtime,tdf,umap
0,hdb,t1+1.1+1.2+1.3+1.4,1,ss50cs50,50,50,1.688348,"rt_vect: 0.7587966918945312, min_df: 4, max_df...","hellinger, rt_umap_tf: 137.97858548164368, rt_..."
1,hdb,t1+1.1+1.2+1.3+1.4,1,ss100cs50,50,100,1.372967,"rt_vect: 0.7587966918945312, min_df: 4, max_df...","hellinger, rt_umap_tf: 137.97858548164368, rt_..."
2,hdb,t1+1.1+1.2+1.3+1.4,1,ss150cs50,50,150,1.591077,"rt_vect: 0.7587966918945312, min_df: 4, max_df...","hellinger, rt_umap_tf: 137.97858548164368, rt_..."
3,hdb,t1+1.1+1.2+1.3+1.4,1,ss200cs50,50,200,1.969689,"rt_vect: 0.7587966918945312, min_df: 4, max_df...","hellinger, rt_umap_tf: 137.97858548164368, rt_..."


,type,testset,version,name,hdb_minclust,hdb_minsamp,runtime,tdf,umap
0,hdb,t1+1.1+1.2+1.3+1.4,1,ss50cs50,50,50,1.688348,"rt_vect: 0.7587966918945312, min_df: 4, max_df...","hellinger, rt_umap_tf: 137.97858548164368, rt_..."
1,hdb,t1+1.1+1.2+1.3+1.4,1,ss100cs50,50,100,1.372967,"rt_vect: 0.7587966918945312, min_df: 4, max_df...","hellinger, rt_umap_tf: 137.97858548164368, rt_..."
2,hdb,t1+1.1+1.2+1.3+1.4,1,ss150cs50,50,150,1.591077,"rt_vect: 0.7587966918945312, min_df: 4, max_df...","hellinger, rt_umap_tf: 137.97858548164368, rt_..."
3,hdb,t1+1.1+1.2+1.3+1.4,1,ss200cs50,50,200,1.969689,"rt_vect: 0.7587966918945312, min_df: 4, max_df...","hellinger, rt_umap_tf: 137.97858548164368, rt_..."
4,hdb,t1+1.1+1.2+1.3+1.4,1,ss250cs50,50,250,2.129846,"rt_vect: 0.7587966918945312, min_df: 4, max_df...","hellinger, rt_umap_tf: 137.97858548164368, rt_..."


...

,type,testset,version,name,hdb_minclust,hdb_minsamp,runtime,tdf,umap
0,hdb,t1+1.1+1.2+1.3+1.4,1,ss50cs50,50,50,1.688348,"rt_vect: 0.7587966918945312, min_df: 4, max_df...","hellinger, rt_umap_tf: 137.97858548164368, rt_..."
1,hdb,t1+1.1+1.2+1.3+1.4,1,ss100cs50,50,100,1.372967,"rt_vect: 0.7587966918945312, min_df: 4, max_df...","hellinger, rt_umap_tf: 137.97858548164368, rt_..."
2,hdb,t1+1.1+1.2+1.3+1.4,1,ss150cs50,50,150,1.591077,"rt_vect: 0.7587966918945312, min_df: 4, max_df...","hellinger, rt_umap_tf: 137.97858548164368, rt_..."
3,hdb,t1+1.1+1.2+1.3+1.4,1,ss200cs50,50,200,1.969689,"rt_vect: 0.7587966918945312, min_df: 4, max_df...","hellinger, rt_umap_tf: 137.97858548164368, rt_..."
4,hdb,t1+1.1+1.2+1.3+1.4,1,ss250cs50,50,250,2.129846,"rt_vect: 0.7587966918945312, min_df: 4, max_df...","hellinger, rt_umap_tf: 137.97858548164368, rt_..."
...,...,...,...,...,...,...,...,...,...
72,hdb,t1+1.1+1.2+1.3+1.4,1,ss50cs450,450,50,1.277472,"rt_vect: 0.7587966918945312, min_df: 4, max_df...","hellinger, rt_umap_tf: 137.97858548164368, rt_..."
73,hdb,t1+1.1+1.2+1.3+1.4,1,ss100cs450,450,100,1.374296,"rt_vect: 0.7587966918945312, min_df: 4, max_df...","hellinger, rt_umap_tf: 137.97858548164368, rt_..."
74,hdb,t1+1.1+1.2+1.3+1.4,1,ss150cs450,450,150,1.528259,"rt_vect: 0.7587966918945312, min_df: 4, max_df...","hellinger, rt_umap_tf: 137.97858548164368, rt_..."
75,hdb,t1+1.1+1.2+1.3+1.4,1,ss200cs450,450,200,1.741901,"rt_vect: 0.7587966918945312, min_df: 4, max_df...","hellinger, rt_umap_tf: 137.97858548164368, rt_..."


,type,testset,version,name,hdb_minclust,hdb_minsamp,runtime,tdf,umap
0,hdb,t1+1.1+1.2+1.3+1.4,1,ss50cs50,50,50,1.688348,"rt_vect: 0.7587966918945312, min_df: 4, max_df...","hellinger, rt_umap_tf: 137.97858548164368, rt_..."
1,hdb,t1+1.1+1.2+1.3+1.4,1,ss100cs50,50,100,1.372967,"rt_vect: 0.7587966918945312, min_df: 4, max_df...","hellinger, rt_umap_tf: 137.97858548164368, rt_..."
2,hdb,t1+1.1+1.2+1.3+1.4,1,ss150cs50,50,150,1.591077,"rt_vect: 0.7587966918945312, min_df: 4, max_df...","hellinger, rt_umap_tf: 137.97858548164368, rt_..."
3,hdb,t1+1.1+1.2+1.3+1.4,1,ss200cs50,50,200,1.969689,"rt_vect: 0.7587966918945312, min_df: 4, max_df...","hellinger, rt_umap_tf: 137.97858548164368, rt_..."
4,hdb,t1+1.1+1.2+1.3+1.4,1,ss250cs50,50,250,2.129846,"rt_vect: 0.7587966918945312, min_df: 4, max_df...","hellinger, rt_umap_tf: 137.97858548164368, rt_..."
...,...,...,...,...,...,...,...,...,...
73,hdb,t1+1.1+1.2+1.3+1.4,1,ss100cs450,450,100,1.374296,"rt_vect: 0.7587966918945312, min_df: 4, max_df...","hellinger, rt_umap_tf: 137.97858548164368, rt_..."
74,hdb,t1+1.1+1.2+1.3+1.4,1,ss150cs450,450,150,1.528259,"rt_vect: 0.7587966918945312, min_df: 4, max_df...","hellinger, rt_umap_tf: 137.97858548164368, rt_..."
75,hdb,t1+1.1+1.2+1.3+1.4,1,ss200cs450,450,200,1.741901,"rt_vect: 0.7587966918945312, min_df: 4, max_df...","hellinger, rt_umap_tf: 137.97858548164368, rt_..."
76,hdb,t1+1.1+1.2+1.3+1.4,1,ss250cs450,450,250,1.988653,"rt_vect: 0.7587966918945312, min_df: 4, max_df...","hellinger, rt_umap_tf: 137.97858548164368, rt_..."


,type,testset,version,name,hdb_minclust,hdb_minsamp,runtime,tdf,umap
0,hdb,t1+1.1+1.2+1.3+1.4,1,ss50cs50,50,50,1.688348,"rt_vect: 0.7587966918945312, min_df: 4, max_df...","hellinger, rt_umap_tf: 137.97858548164368, rt_..."
1,hdb,t1+1.1+1.2+1.3+1.4,1,ss100cs50,50,100,1.372967,"rt_vect: 0.7587966918945312, min_df: 4, max_df...","hellinger, rt_umap_tf: 137.97858548164368, rt_..."
2,hdb,t1+1.1+1.2+1.3+1.4,1,ss150cs50,50,150,1.591077,"rt_vect: 0.7587966918945312, min_df: 4, max_df...","hellinger, rt_umap_tf: 137.97858548164368, rt_..."
3,hdb,t1+1.1+1.2+1.3+1.4,1,ss200cs50,50,200,1.969689,"rt_vect: 0.7587966918945312, min_df: 4, max_df...","hellinger, rt_umap_tf: 137.97858548164368, rt_..."
4,hdb,t1+1.1+1.2+1.3+1.4,1,ss250cs50,50,250,2.129846,"rt_vect: 0.7587966918945312, min_df: 4, max_df...","hellinger, rt_umap_tf: 137.97858548164368, rt_..."
...,...,...,...,...,...,...,...,...,...
74,hdb,t1+1.1+1.2+1.3+1.4,1,ss150cs450,450,150,1.528259,"rt_vect: 0.7587966918945312, min_df: 4, max_df...","hellinger, rt_umap_tf: 137.97858548164368, rt_..."
75,hdb,t1+1.1+1.2+1.3+1.4,1,ss200cs450,450,200,1.741901,"rt_vect: 0.7587966918945312, min_df: 4, max_df...","hellinger, rt_umap_tf: 137.97858548164368, rt_..."
76,hdb,t1+1.1+1.2+1.3+1.4,1,ss250cs450,450,250,1.988653,"rt_vect: 0.7587966918945312, min_df: 4, max_df...","hellinger, rt_umap_tf: 137.97858548164368, rt_..."
77,hdb,t1+1.1+1.2+1.3+1.4,1,ss300cs450,450,300,2.323644,"rt_vect: 0.7587966918945312, min_df: 4, max_df...","hellinger, rt_umap_tf: 137.97858548164368, rt_..."


,type,testset,version,name,hdb_minclust,hdb_minsamp,runtime,tdf,umap
0,hdb,t1+1.1+1.2+1.3+1.4,1,ss50cs50,50,50,1.688348,"rt_vect: 0.7587966918945312, min_df: 4, max_df...","hellinger, rt_umap_tf: 137.97858548164368, rt_..."
1,hdb,t1+1.1+1.2+1.3+1.4,1,ss100cs50,50,100,1.372967,"rt_vect: 0.7587966918945312, min_df: 4, max_df...","hellinger, rt_umap_tf: 137.97858548164368, rt_..."
2,hdb,t1+1.1+1.2+1.3+1.4,1,ss150cs50,50,150,1.591077,"rt_vect: 0.7587966918945312, min_df: 4, max_df...","hellinger, rt_umap_tf: 137.97858548164368, rt_..."
3,hdb,t1+1.1+1.2+1.3+1.4,1,ss200cs50,50,200,1.969689,"rt_vect: 0.7587966918945312, min_df: 4, max_df...","hellinger, rt_umap_tf: 137.97858548164368, rt_..."
4,hdb,t1+1.1+1.2+1.3+1.4,1,ss250cs50,50,250,2.129846,"rt_vect: 0.7587966918945312, min_df: 4, max_df...","hellinger, rt_umap_tf: 137.97858548164368, rt_..."
...,...,...,...,...,...,...,...,...,...
75,hdb,t1+1.1+1.2+1.3+1.4,1,ss200cs450,450,200,1.741901,"rt_vect: 0.7587966918945312, min_df: 4, max_df...","hellinger, rt_umap_tf: 137.97858548164368, rt_..."
76,hdb,t1+1.1+1.2+1.3+1.4,1,ss250cs450,450,250,1.988653,"rt_vect: 0.7587966918945312, min_df: 4, max_df...","hellinger, rt_umap_tf: 137.97858548164368, rt_..."
77,hdb,t1+1.1+1.2+1.3+1.4,1,ss300cs450,450,300,2.323644,"rt_vect: 0.7587966918945312, min_df: 4, max_df...","hellinger, rt_umap_tf: 137.97858548164368, rt_..."
78,hdb,t1+1.1+1.2+1.3+1.4,1,ss350cs450,450,350,2.411070,"rt_vect: 0.7587966918945312, min_df: 4, max_df...","hellinger, rt_umap_tf: 137.97858548164368, rt_..."


,type,testset,version,name,hdb_minclust,hdb_minsamp,runtime,tdf,umap
0,hdb,t1+1.1+1.2+1.3+1.4,1,ss50cs50,50,50,1.688348,"rt_vect: 0.7587966918945312, min_df: 4, max_df...","hellinger, rt_umap_tf: 137.97858548164368, rt_..."
1,hdb,t1+1.1+1.2+1.3+1.4,1,ss100cs50,50,100,1.372967,"rt_vect: 0.7587966918945312, min_df: 4, max_df...","hellinger, rt_umap_tf: 137.97858548164368, rt_..."
2,hdb,t1+1.1+1.2+1.3+1.4,1,ss150cs50,50,150,1.591077,"rt_vect: 0.7587966918945312, min_df: 4, max_df...","hellinger, rt_umap_tf: 137.97858548164368, rt_..."
3,hdb,t1+1.1+1.2+1.3+1.4,1,ss200cs50,50,200,1.969689,"rt_vect: 0.7587966918945312, min_df: 4, max_df...","hellinger, rt_umap_tf: 137.97858548164368, rt_..."
4,hdb,t1+1.1+1.2+1.3+1.4,1,ss250cs50,50,250,2.129846,"rt_vect: 0.7587966918945312, min_df: 4, max_df...","hellinger, rt_umap_tf: 137.97858548164368, rt_..."
...,...,...,...,...,...,...,...,...,...
76,hdb,t1+1.1+1.2+1.3+1.4,1,ss250cs450,450,250,1.988653,"rt_vect: 0.7587966918945312, min_df: 4, max_df...","hellinger, rt_umap_tf: 137.97858548164368, rt_..."
77,hdb,t1+1.1+1.2+1.3+1.4,1,ss300cs450,450,300,2.323644,"rt_vect: 0.7587966918945312, min_df: 4, max_df...","hellinger, rt_umap_tf: 137.97858548164368, rt_..."
78,hdb,t1+1.1+1.2+1.3+1.4,1,ss350cs450,450,350,2.411070,"rt_vect: 0.7587966918945312, min_df: 4, max_df...","hellinger, rt_umap_tf: 137.97858548164368, rt_..."
79,hdb,t1+1.1+1.2+1.3+1.4,1,ss400cs450,450,400,2.629624,"rt_vect: 0.7587966918945312, min_df: 4, max_df...","hellinger, rt_umap_tf: 137.97858548164368, rt_..."


In [26]:
hdb_labels_results

,name,ss50cs50,ss100cs50,ss150cs50,ss200cs50,ss250cs50,ss300cs50,ss350cs50,ss400cs50,ss450cs50,...,ss450cs400,ss50cs450,ss100cs450,ss150cs450,ss200cs450,ss250cs450,ss300cs450,ss350cs450,ss400cs450,ss450cs450
0,thingsym/multi-device-switcher,8,-1,-1,-1,-1,-1,-1,2,2,...,2,3,2,-1,-1,-1,-1,-1,2,2
1,redlink-gmbh/solr-compound-word-filter,110,-1,-1,-1,-1,-1,14,2,2,...,2,3,2,3,3,2,2,3,2,2
2,xswlme/Delivery-Availability-Checker,112,-1,-1,-1,-1,-1,14,2,2,...,2,3,2,3,3,2,2,3,2,2
3,eastsss/ErrorDispatching,-1,-1,-1,-1,-1,-1,14,2,2,...,2,3,2,3,3,2,2,3,2,2
4,ahmadhuzaifa/skinskan,-1,74,50,43,32,30,29,2,2,...,2,3,2,3,3,2,2,3,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69812,waqasbhatti/astrobase,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
69813,hpcc-systems/vscode-ecl,-1,-1,-1,-1,-1,-1,-1,2,2,...,2,3,2,3,3,2,2,3,2,2
69814,MarcoPon/BlockHashLoc,-1,-1,63,54,41,35,14,2,2,...,2,3,2,3,3,2,2,3,2,2
69815,bhagatabhijeet/bhagatabhijeet,-1,-1,-1,-1,-1,-1,-1,2,2,...,2,3,2,3,3,2,2,3,2,2


In [27]:
print("saving results to disk")
try:
    old_results = from_disk(f'results/{cust_dir}/hdb_results')
    concat_results = pd.concat([old_results, results])
    concat_results.drop_duplicates(inplace=True)
    to_disk_results(concat_results, f'hdb_results', subdir=f"{cust_dir}")
except FileNotFoundError:
    to_disk_results(results, f'hdb_results', subdir=f"{cust_dir}")
to_disk_results(hdb_labels_results, f'hdb_labels_results_{t_name}', subdir=f"{cust_dir}")

saving results to disk
reading /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/results/ar1//hdb_results.pkl
writing /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/results/ar1//hdb_results.pkl
writing /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/results/ar1//hdb_labels_results_t1+1.1+1.2+1.3+1.4.pkl


In [28]:

umap_embedding['topics'] = testset['topics']
umap_clusterable_embedding['topics'] = testset['topics']

In [29]:
import sklearn.cluster as cluster

def clusterapply_kmeans(testset, clustersize, runner_name, opt_comment=""):
    # Perform k-means clustering
    
    kmeans = cluster.KMeans(n_clusters=clustersize, random_state=42)
        
    if('y' in testset.columns):
        #print("assuming umap reduced testset here")
        testset = testset.copy()
        dropping_not_connected = len(testset[testset['y'].isnull()])
        testset = testset[testset['y'].notna()]
        cr_start()
        kmeans.fit(testset[['x', 'y']].to_numpy())
        runtime_kmeans = cr_getrt()
        opt_comment="-umapt"
    else:
        dropping_not_connected = 0
        cr_start()
        kmeans.fit(word_doc_matrix)
        runtime_kmeans = cr_getrt()

    
    stats = {'type': f"kmeans{opt_comment}",
             'testset': t_name,
             'version': td_version,
             'size': clustersize
            }
    
    detail_stats = {'tdf': f"rt_vect: {runtime_vectorizer}, min_df: {tdidf_min_df}, max_df: {tdidf_max_df}, stopwords: {use_stopwords}, extended stopwords: {use_extended_stopwords}",
                'runtime': runtime_kmeans
               }
    
    if('y' in testset.columns):
        detail_stats['umap'] = f"{umap_metric}, rt_umap_tf: {runtime_umap_transform}, rt_umap: {runtime_umap}, ncomp: {umap_ncomponents}, rs: {umap_random_state}, min_dist: {umap_min_dist}, no_neigh: {umap_n_neighbors}", #umap not important here
        stats['no_con'] = dropping_not_connected

    prediction = stats
    prediction['name'] = runner_name
    #prediction.update(max_fscore(my_results[[tag, 'clabel']].apply(tuple, axis=1).to_dict()))
    #prediction.update({'tot_clust': total_clusters, 'no_lab':missing_clabel})
    prediction.update(detail_stats)
    res.append(prediction.copy())
        
    display(pd.DataFrame.from_dict(res))
    return kmeans.labels_
    #testset.drop(['cluster_run'], axis=1)


In [30]:
res = []

kmeans_labels_results = pd.DataFrame()
kmeans_labels_results['name'] = umap_embedding[umap_embedding['y'].notna()]['name'].copy()

for i in range(min_cluster, max_cluster, clust_int): # 10, 60
    runner_name = f'{i}'
    kmeans_labels_results[runner_name] = clusterapply_kmeans(umap_embedding, i, runner_name)

results = pd.DataFrame.from_dict(res)
res = []

#k_means_df[['x', 'y']].to_numpy(), 
#word_doc_matrix



,type,testset,version,size,no_con,name,tdf,runtime,umap
0,kmeans-umapt,t1+1.1+1.2+1.3+1.4,1,10,777,10,"rt_vect: 0.7587966918945312, min_df: 4, max_df...",0.547558,"(hellinger, rt_umap_tf: 137.97858548164368, rt..."


,type,testset,version,size,no_con,name,tdf,runtime,umap
0,kmeans-umapt,t1+1.1+1.2+1.3+1.4,1,10,777,10,"rt_vect: 0.7587966918945312, min_df: 4, max_df...",0.547558,"(hellinger, rt_umap_tf: 137.97858548164368, rt..."
1,kmeans-umapt,t1+1.1+1.2+1.3+1.4,1,13,777,13,"rt_vect: 0.7587966918945312, min_df: 4, max_df...",0.516245,"(hellinger, rt_umap_tf: 137.97858548164368, rt..."


,type,testset,version,size,no_con,name,tdf,runtime,umap
0,kmeans-umapt,t1+1.1+1.2+1.3+1.4,1,10,777,10,"rt_vect: 0.7587966918945312, min_df: 4, max_df...",0.547558,"(hellinger, rt_umap_tf: 137.97858548164368, rt..."
1,kmeans-umapt,t1+1.1+1.2+1.3+1.4,1,13,777,13,"rt_vect: 0.7587966918945312, min_df: 4, max_df...",0.516245,"(hellinger, rt_umap_tf: 137.97858548164368, rt..."
2,kmeans-umapt,t1+1.1+1.2+1.3+1.4,1,16,777,16,"rt_vect: 0.7587966918945312, min_df: 4, max_df...",0.617096,"(hellinger, rt_umap_tf: 137.97858548164368, rt..."


,type,testset,version,size,no_con,name,tdf,runtime,umap
0,kmeans-umapt,t1+1.1+1.2+1.3+1.4,1,10,777,10,"rt_vect: 0.7587966918945312, min_df: 4, max_df...",0.547558,"(hellinger, rt_umap_tf: 137.97858548164368, rt..."
1,kmeans-umapt,t1+1.1+1.2+1.3+1.4,1,13,777,13,"rt_vect: 0.7587966918945312, min_df: 4, max_df...",0.516245,"(hellinger, rt_umap_tf: 137.97858548164368, rt..."
2,kmeans-umapt,t1+1.1+1.2+1.3+1.4,1,16,777,16,"rt_vect: 0.7587966918945312, min_df: 4, max_df...",0.617096,"(hellinger, rt_umap_tf: 137.97858548164368, rt..."
3,kmeans-umapt,t1+1.1+1.2+1.3+1.4,1,19,777,19,"rt_vect: 0.7587966918945312, min_df: 4, max_df...",0.756519,"(hellinger, rt_umap_tf: 137.97858548164368, rt..."


,type,testset,version,size,no_con,name,tdf,runtime,umap
0,kmeans-umapt,t1+1.1+1.2+1.3+1.4,1,10,777,10,"rt_vect: 0.7587966918945312, min_df: 4, max_df...",0.547558,"(hellinger, rt_umap_tf: 137.97858548164368, rt..."
1,kmeans-umapt,t1+1.1+1.2+1.3+1.4,1,13,777,13,"rt_vect: 0.7587966918945312, min_df: 4, max_df...",0.516245,"(hellinger, rt_umap_tf: 137.97858548164368, rt..."
2,kmeans-umapt,t1+1.1+1.2+1.3+1.4,1,16,777,16,"rt_vect: 0.7587966918945312, min_df: 4, max_df...",0.617096,"(hellinger, rt_umap_tf: 137.97858548164368, rt..."
3,kmeans-umapt,t1+1.1+1.2+1.3+1.4,1,19,777,19,"rt_vect: 0.7587966918945312, min_df: 4, max_df...",0.756519,"(hellinger, rt_umap_tf: 137.97858548164368, rt..."
4,kmeans-umapt,t1+1.1+1.2+1.3+1.4,1,22,777,22,"rt_vect: 0.7587966918945312, min_df: 4, max_df...",0.924909,"(hellinger, rt_umap_tf: 137.97858548164368, rt..."


,type,testset,version,size,no_con,name,tdf,runtime,umap
0,kmeans-umapt,t1+1.1+1.2+1.3+1.4,1,10,777,10,"rt_vect: 0.7587966918945312, min_df: 4, max_df...",0.547558,"(hellinger, rt_umap_tf: 137.97858548164368, rt..."
1,kmeans-umapt,t1+1.1+1.2+1.3+1.4,1,13,777,13,"rt_vect: 0.7587966918945312, min_df: 4, max_df...",0.516245,"(hellinger, rt_umap_tf: 137.97858548164368, rt..."
2,kmeans-umapt,t1+1.1+1.2+1.3+1.4,1,16,777,16,"rt_vect: 0.7587966918945312, min_df: 4, max_df...",0.617096,"(hellinger, rt_umap_tf: 137.97858548164368, rt..."
3,kmeans-umapt,t1+1.1+1.2+1.3+1.4,1,19,777,19,"rt_vect: 0.7587966918945312, min_df: 4, max_df...",0.756519,"(hellinger, rt_umap_tf: 137.97858548164368, rt..."
4,kmeans-umapt,t1+1.1+1.2+1.3+1.4,1,22,777,22,"rt_vect: 0.7587966918945312, min_df: 4, max_df...",0.924909,"(hellinger, rt_umap_tf: 137.97858548164368, rt..."
5,kmeans-umapt,t1+1.1+1.2+1.3+1.4,1,25,777,25,"rt_vect: 0.7587966918945312, min_df: 4, max_df...",0.989473,"(hellinger, rt_umap_tf: 137.97858548164368, rt..."


,type,testset,version,size,no_con,name,tdf,runtime,umap
0,kmeans-umapt,t1+1.1+1.2+1.3+1.4,1,10,777,10,"rt_vect: 0.7587966918945312, min_df: 4, max_df...",0.547558,"(hellinger, rt_umap_tf: 137.97858548164368, rt..."
1,kmeans-umapt,t1+1.1+1.2+1.3+1.4,1,13,777,13,"rt_vect: 0.7587966918945312, min_df: 4, max_df...",0.516245,"(hellinger, rt_umap_tf: 137.97858548164368, rt..."
2,kmeans-umapt,t1+1.1+1.2+1.3+1.4,1,16,777,16,"rt_vect: 0.7587966918945312, min_df: 4, max_df...",0.617096,"(hellinger, rt_umap_tf: 137.97858548164368, rt..."
3,kmeans-umapt,t1+1.1+1.2+1.3+1.4,1,19,777,19,"rt_vect: 0.7587966918945312, min_df: 4, max_df...",0.756519,"(hellinger, rt_umap_tf: 137.97858548164368, rt..."
4,kmeans-umapt,t1+1.1+1.2+1.3+1.4,1,22,777,22,"rt_vect: 0.7587966918945312, min_df: 4, max_df...",0.924909,"(hellinger, rt_umap_tf: 137.97858548164368, rt..."
5,kmeans-umapt,t1+1.1+1.2+1.3+1.4,1,25,777,25,"rt_vect: 0.7587966918945312, min_df: 4, max_df...",0.989473,"(hellinger, rt_umap_tf: 137.97858548164368, rt..."
6,kmeans-umapt,t1+1.1+1.2+1.3+1.4,1,28,777,28,"rt_vect: 0.7587966918945312, min_df: 4, max_df...",1.154761,"(hellinger, rt_umap_tf: 137.97858548164368, rt..."


In [31]:
kmeans_labels_results

,name,10,13,16,19,22,25,28
0,thingsym/multi-device-switcher,2,3,9,7,8,17,20
1,redlink-gmbh/solr-compound-word-filter,1,11,3,5,3,15,23
2,xswlme/Delivery-Availability-Checker,4,9,15,2,1,20,11
3,eastsss/ErrorDispatching,4,9,15,15,16,20,16
4,ahmadhuzaifa/skinskan,0,6,0,8,6,10,0
...,...,...,...,...,...,...,...,...
69812,waqasbhatti/astrobase,9,1,8,11,14,24,27
69813,hpcc-systems/vscode-ecl,4,9,2,0,1,20,11
69814,MarcoPon/BlockHashLoc,1,11,3,5,3,15,23
69815,bhagatabhijeet/bhagatabhijeet,7,4,10,2,1,7,11


In [32]:
print("saving results to disk")
try:
    old_results = from_disk(f'results/{cust_dir}/kmeans-umap_results')
    concat_results = pd.concat([old_results, results])
    concat_results.drop_duplicates(inplace=True)
    to_disk_results(concat_results, f'kmeans-umap_results', subdir=f"{cust_dir}")
except FileNotFoundError:
    to_disk_results(results, f'kmeans-umap_results', subdir=f"{cust_dir}")
    
to_disk_results(kmeans_labels_results, f'kmeans-umap_labels_results_{t_name}', subdir=f"{cust_dir}")


saving results to disk
reading /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/results/ar1//kmeans-umap_results.pkl
writing /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/results/ar1//kmeans-umap_results.pkl
writing /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/results/ar1//kmeans-umap_labels_results_t1+1.1+1.2+1.3+1.4.pkl


In [33]:
res = []

kmeans_labels_results = pd.DataFrame()
kmeans_labels_results['name'] = testset['name'].copy()


for i in range(10, 16, 3): # 10 60
    runner_name = f'{i}'
    kmeans_labels_results[runner_name] = clusterapply_kmeans(testset, i, runner_name)

results = pd.DataFrame.from_dict(res)
res = []

#k_means_df[['x', 'y']].to_numpy(), 
#word_doc_matrix

,type,testset,version,size,name,tdf,runtime
0,kmeans,t1+1.1+1.2+1.3+1.4,1,10,10,"rt_vect: 0.7587966918945312, min_df: 4, max_df...",2.550253


,type,testset,version,size,name,tdf,runtime
0,kmeans,t1+1.1+1.2+1.3+1.4,1,10,10,"rt_vect: 0.7587966918945312, min_df: 4, max_df...",2.550253
1,kmeans,t1+1.1+1.2+1.3+1.4,1,13,13,"rt_vect: 0.7587966918945312, min_df: 4, max_df...",3.596661


In [34]:
kmeans_labels_results

,name,10,13
0,thingsym/multi-device-switcher,0,9
1,redlink-gmbh/solr-compound-word-filter,0,9
2,xswlme/Delivery-Availability-Checker,0,9
3,eastsss/ErrorDispatching,0,9
4,ahmadhuzaifa/skinskan,2,3
...,...,...,...
69812,waqasbhatti/astrobase,0,9
69813,hpcc-systems/vscode-ecl,0,9
69814,MarcoPon/BlockHashLoc,0,9
69815,bhagatabhijeet/bhagatabhijeet,0,9


In [35]:
print("saving results to disk")

try:
    old_results = from_disk(f'results/{cust_dir}/kmeans_results')
    concat_results = pd.concat([old_results, results])
    concat_results.drop_duplicates(inplace=True)
    to_disk_results(concat_results, f'kmeans_results', subdir=f"{cust_dir}")
except FileNotFoundError:
    to_disk_results(results, f'kmeans_results', subdir=f"{cust_dir}")
    
to_disk_results(kmeans_labels_results, f'kmeans_labels_results_{t_name}', subdir=f"{cust_dir}")


saving results to disk
reading /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/results/ar1//kmeans_results.pkl
writing /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/results/ar1//kmeans_results.pkl
writing /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/results/ar1//kmeans_labels_results_t1+1.1+1.2+1.3+1.4.pkl
